In [2]:
import pickle
import pandas as pd
import os
import setup

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_tools.rnn_classes as rnn_classes
from rnn_tools.raceDB import build_dataset
import importlib
import datetime
from rnn_tools.model_saver import model_saver, model_saver_wandb
import rnn_tools.training_testing_gru as training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence
import rnn_tools.training_testing_gru_double as training_testing_gru_double
import rnn_tools.training_testing_lstm as training_testing_lstm
from goto_conversion import goto_conversion



In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
torch.set_default_tensor_type(torch.FloatTensor)

c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [5]:
torch.__version__

'2.1.1'

In [6]:
['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'margin_time_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']

['dist_last__1',
 'box_last__1',
 'speed_avg_1',
 'split_speed_v1_1',
 'split_speed_avg_1',
 'split_margin_avg_1',
 'margin_avg_1',
 'margin_time_avg_1',
 'RunHomeTime_1',
 'run_home_speed_1',
 'run_home_speed_v1_1',
 'first_out_avg_1',
 'pos_out_avg_1',
 'post_change_avg_1',
 'races_1',
 'wins_1',
 'wins_last_1',
 'weight_',
 'min_time_',
 'min_split_time_',
 'min_split_time_v1',
 'last_start_price',
 'last_start_prob']

In [7]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [8]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [9]:
tracks = ['Sandown Park',
 'The Meadows',
 'Warragul',
 'Geelong',
 'Sale',
 'Ballarat',
 'Bendigo',
 'Cranbourne']

In [10]:
os.getcwd()
# os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["VIC"]
data_file = './data/gru_inputs_simple_kitchen_sink_new.fth'
raceDB = build_dataset(data_file, hidden_size ,state_filter=states, margin_type='boosted_sftmin',v6=True)
raceDB.create_new_weights_v2()

[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'margin_time_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]
(1465899, 25)
26
Latest date = 2023-12-11 00:00:00
size after state filter (434463, 27)
(434463, 27)
Latest date = 2023-12-11 00:00:00
num_features_per_dog=26
___Processing 17493 unique dogs___


100%|██████████| 25/25 [00:00<?, ?it/s]
c:\Users\Nick\Documents\GitHub\grvmodel\python\rnn_tools\rnn_classes.py:391: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  r.prob = torch.tensor((1/(torch.tensor(r.start_prices)+0.0001))/((1/torch.tensor(r.start_prices)).sum()+0.0001),device=self.device)


number of races = 60525, number of unique dogs = 17494
0            (Bendigo, 425.0)
1        (The Meadows, 525.0)
2               (Sale, 440.0)
3            (Bendigo, 425.0)
4           (Ballarat, 450.0)
                 ...         
60520    (Healesville, 300.0)
60521    (Healesville, 350.0)
60522       (Ballarat, 660.0)
60523       (Ballarat, 545.0)
60524       (Ballarat, 450.0)
Length: 60525, dtype: object


In [11]:
races = raceDB.get_race_input(range(1000,1100))

In [12]:
stat_list = ['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'margin_time_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']


In [13]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU V7", config=config,save_code=False):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss_val", summary="min")
      wandb.define_metric("accuracy", summary="max")
      wandb.define_metric("ROI < 30", summary="max")
      wandb.define_metric("relu roi", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)

      stat_mask = [1,1,1]
      for stat in stat_list:

          stat_flag = config[stat]

          stat_mask = stat_mask+[stat_flag]

      print(stat_mask)
      stat_mask = torch.tensor(stat_mask).type(torch.bool).to(device)

      print(stat_mask)
      print(stat_mask.shape)

      if 'batch_days' in config.keys():
        pass
        # raceDB.create_test_split_date(config['training_date_end'],val_date='2023-12-01')
        # raceDB.create_dogs_test_split_date()
        # raceDB.attach_races_to_dog_inputv2() 
        # raceDB.reset_hidden()
        # raceDB.create_batches(batch_days=config['batch_days'], end_date = config['training_date_end'], stat_mask=stat_mask)

      if config['input_type'] == 'basic':
          print('here')
          raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
          raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
          raceDB.batches['packed_v'] = raceDB.batches['packed_v_basic']
          input_size = raceDB.batches['packed_x'][0].data[0].shape[0]

      for race in raceDB.racesDict.values():
          for dog_input in race.dogs:
            dog_input.stats_masked = dog_input.stats.masked_select(stat_mask) 


      print(f"{input_size=}")



      config['stat_mask_tensor'] = torch.tensor(stat_mask, dtype=torch.uint8).to(device)     
    

      


      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.LSTMNetv3_BN_double(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'])
      
      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        print(f"Loading model {prev_model_file}, version {prev_model_version}")
        model_name = prev_model_file
        model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        print(model_data['model_state_dict'].keys())
        # del_keys = ['gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0','layer_norm.weight', 'layer_norm.bias']
        # [model_data['model_state_dict'].pop(k) for k in del_keys]
        model.load_state_dict(model_data['model_state_dict'], strict=False)
        config['parent model'] = prev_model_file
        model = model.to(device)
        optimizer = optim.RAdam(model.parameters(), lr=config['learning_rate'])
        optimizer.load_state_dict(model_data['optim'])
        # optimizer.to(device)
      else:
        optimizer = optim.RAdam(model.parameters(), lr=config['learning_rate'])

      raceDB.to_cuda()
      scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

      criterion = nn.CrossEntropyLoss(reduction='none')

      model = model.to(device)
      print(model)

      # and use them to train the model
      wandb.watch(model, log='all')
      try:
        # train_double_loss_regular(model, dataset, criterion, optimizer, scheduler, config, crit2=custom_l2)
        training_testing_lstm.train_double_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      # dataset.create_hidden_states_dict()
      raceDB.create_hidden_states_dict_v2()
      model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")
      if sweep:
        # raceDB.reset_all_lstm_states
        raceDB.reset_hidden()
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [15]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':128,
                        'stats':raceDB.stats_cols,
                        'races':states,
                        'datafile':data_file,
                        'latest_date':raceDB.latest_date,
                        'input_type':'basic',
                        'num_layers':2,
                        'batch_size': 750,
                        'dropout': 0.3,
                        'epochs': 500,
                        'learning_rate': 0.0001,
                        'optimizer': 'adamW',
                        'f0_layer_size':128,
                        'f1_layer_size':64,
                        'training_date_end':'2021-11-30',
                        'batch_days':365,
                        'dist_last__1': 1,#
                        'box_last__1':1,#
                        'speed_avg_1':1,#
                        'split_speed_v1_1':1,#
                        'split_speed_avg_1':0,
                        'split_margin_avg_1':0,
                        'margin_avg_1':0,
                        'margin_time_avg_1':1,#
                        'RunHomeTime_1':0,
                        'run_home_speed_1':0,
                        'run_home_speed_v1_1':0,
                        'first_out_avg_1':0,
                        'pos_out_avg_1':1,#
                        'post_change_avg_1':1,#
                        'races_1':0,
                        'wins_1':0,
                        'wins_last_1':1,#
                        'weight_':1,#
                        'min_time_':0,
                        'min_split_time_':0, 
                        'min_split_time_v1':1,#
                        'last_start_price':1, #
                        'last_start_prob':0,
            }

In [16]:
wandb_config_static =  {'hidden_size':128,
                        'stats':raceDB.stats_cols,
                        'races':states,
                        'datafile':data_file,
                        'latest_date':raceDB.latest_date,
                        'input_type':'basic',
                        'num_layers':2,
                        'batch_size': 60,
                        'dropout': 0.3,
                        'epochs': 5000,
                        'learning_rate': 0.001,
                        'optimizer': 'adamW',
                        'f0_layer_size':256,
                        'f1_layer_size':128,
                        'training_date_end':'2023-05-01',
                        'batch_days':365,
                        'dist_last__1': 1,#
                        'box_last__1':1,#
                        'speed_avg_1':1,#
                        'split_speed_v1_1':0,#
                        'split_speed_avg_1':1,#
                        'split_margin_avg_1':1,
                        'margin_avg_1':1,#
                        'margin_time_avg_1':1,#
                        'RunHomeTime_1':1,
                        'run_home_speed_1':1,
                        'run_home_speed_v1_1':0,
                        'first_out_avg_1':1,#
                        'pos_out_avg_1':1,#
                        'post_change_avg_1':1,#
                        'races_1':1,
                        'wins_1':1,
                        'wins_last_1':1,
                        'weight_':1,#
                        'min_time_':1,#
                        'min_split_time_':1, 
                        'min_split_time_v1':0,#
                        'last_start_price':0, #
                        'last_start_prob':1,
            }

In [23]:
importlib.reload(training_testing_gru)
importlib.reload(training_testing_lstm)
importlib.reload(training_testing_gru_double)
importlib.reload(rnn_classes)

<module 'rnn_tools.rnn_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\python\\rnn_tools\\rnn_classes.py'>

In [18]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])

In [19]:
torch.cuda.empty_cache()

In [20]:
raceDB.race_prices_to_prob()

In [1]:
raceDB.latest_date

NameError: name 'raceDB' is not defined

In [24]:
WANDB_MODE='offline'
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

{'hidden_size': 128, 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'margin_time_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]", 'races': ['VIC'], 'datafile': './data/gru_inputs_simple_kitchen_sink_new.fth', 'latest_date': '2023-12-11T00:00:00', 'input_type': 'basic', 'num_layers': 2, 'batch_size': 60, 'dropout': 0.3, 'epochs': 5000, 'learning_rate': 0.001, 'optimizer': 'adamW', 'f0_layer_size': 256, 'f1_layer_size': 128, 'training_date_end': '2023-05-01', 'batch_days': 365, 'dist_last__1': 1, 'box_last__1': 1, 'speed_avg_1': 1, 'split_speed_v1_1': 0, 'split_speed_avg_1': 1, 'split_margin_avg_1': 1, 'margin_avg_1': 1, 'margin_time_avg_1': 1, 'RunHomeTime_1': 1, 'run_home_speed_1': 1

100%|██████████| 17494/17494 [00:06<00:00, 2880.62it/s]


In [45]:
128*8+70

1094

In [ ]:
criterion = nn.CrossEntropyLoss(reduction='none')

In [ ]:
training_testing_gru_double.test_model_v3(model,raceDB,criterion)

You must call wandb.init() before wandb.log()


Error: You must call wandb.init() before wandb.log()

In [ ]:
sweep_config = {"method": "grid"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "batch_days": {"values": [90,180,365,550,10000]},
    "f0_layer_size": {"values": [128]},
    "f1_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "input_type": {"values": ['basic']},
    "num_layers": {"values": [2]},
    'hidden_size':{'values':[128]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "stats":{"value": raceDB.stats_cols},
    "races":{"value": states},
    "batch_size":{"value": 10},
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [1000]},
        "validation_split": {"value": 0.1},
        "training_date_end": {"values": 
                                [
                                '2023-04-30',
                                '2023-02-28',
                                '2022-11-30',
                                '2021-11-30'
                                ]},
        "learning_rate": {"values": [0.0003, 0.0001]},
        "label_smoothing": {"values": [0.01,0.0]},
        "loss": {
            "values": [ "CEL"],
        },
    'dist_last__1': {'values': [0, 1]},
    'box_last__1': {'values': [0, 1]},
    'speed_avg_1': {'values': [0, 1]},
    'split_speed_v1_1': {'values': [0, 1]},
    'split_speed_avg_1': {'values': [0, 1]},
    'split_margin_avg_1': {'values': [0, 1]},
    'margin_avg_1': {'values': [0, 1]},
    'margin_time_avg_1': {'values': [0, 1]},
    'RunHomeTime_1': {'values': [0, 1]},
    'run_home_speed_1': {'values': [0, 1]},
    'run_home_speed_v1_1': {'values': [0, 1]},
    'first_out_avg_1': {'values': [0, 1]},
    'pos_out_avg_1': {'values': [0, 1]},
    'post_change_avg_1': {'values': [0, 1]},
    'races_1': {'values': [0, 1]},
    'wins_1': {'values': [0, 1]},
    'wins_last_1': {'values': [0, 1]},
    'weight_': {'values': [0, 1]},
    'min_time_': {'values': [0, 1]},
    'min_split_time_': {'values': [0, 1]},
    'min_split_time_v1': {'values': [0, 1]},
    'last_start_price': {'values': [0, 1]},
    'last_start_prob': {'values': [0, 1]}
}
)

import pprint

pprint.pprint(sweep_config)


sweep_config

{'method': 'grid',
 'metric': {'goal': 'maximize', 'name': 'ROI < 30'},
 'parameters': {'RunHomeTime_1': {'values': [0, 1]},
                'batch_days': {'values': [90, 180, 365, 550, 10000]},
                'batch_size': {'value': 10},
                'box_last__1': {'values': [0, 1]},
                'dist_last__1': {'values': [0, 1]},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [500]},
                'f0_layer_size': {'values': [128]},
                'f1_layer_size': {'values': [64]},
                'first_out_avg_1': {'values': [0, 1]},
                'hidden_size': {'values': [128]},
                'input_type': {'values': ['basic']},
                'label_smoothing': {'values': [0.01, 0.0]},
                'last_start_price': {'values': [0, 1]},
                'last_start_prob': {'values': [0, 1]},
                'learning_rate': {'values': [0.0003, 0.0001]},
                'len_data': {'value': 19151},
                'loss': {

{'method': 'grid',
 'metric': {'name': 'ROI < 30', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'batch_days': {'values': [90, 180, 365, 550, 10000]},
  'f0_layer_size': {'values': [128]},
  'f1_layer_size': {'values': [64]},
  'dropout': {'values': [0.3]},
  'input_type': {'values': ['basic']},
  'num_layers': {'values': [2]},
  'hidden_size': {'values': [128]},
  'len_data': {'value': 19151},
  'stats': {'value': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'margin_time_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]"},
  'races': {'value': ['NZ']},
  'batch_size': {'value': 10},
  'epochs': {'values': [500]},
  'validation_split': {'value': 0.1},
  'training_date

In [ ]:
# wandb.agent('x9mz21wl', function=model_pipeline, count=100, project='NEW GRU-sweeps')

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Variable Sweeps")
# CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=1)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Network error (ReadTimeout), entering retry loop.
